In [1]:
# link to Michael Nielsen webpage: http://neuralnetworksanddeeplearning.com/
# this needs python2.7, some part not compatible with python3, i have worked to make it work in python3
# i ran this on devbox01, which has GeForce GTX TITAN X GUPS
# run this in this directory: /projects/trans_scratch/validations/workspace/szong/deep_learning/neural-networks-and-deep-learning/src/

In [2]:
# add this package path to system path so that python can find them
import sys
sys.path.append('/projects/trans_scratch/validations/workspace/szong/deep_learning/neural-networks-and-deep-learning/src/')

In [3]:
import mnist_loader
import network
import pandas as pd
from sklearn import svm
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
!which python

/projects/da_workspace/software/miniconda/envs/dlpy3/bin/python


In [5]:
# GPU
import torch
torch.cuda.is_available()
torch.cuda.current_device()
torch.cuda.device(0)
torch.cuda.device_count()
torch.cuda.get_device_name(0)


True

0

4

'GeForce GTX TITAN X'

In [6]:
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

In [7]:
# data is a list of list of tuple of 
len(training_data)
type(training_data)
# training_data
training_data[0][0].shape
training_data[0][1].shape

training_data[0][0].shape

50000

list

(784, 1)

(10, 1)

(784, 1)

input format is list of tuples, every tuple has 784x1 np array (img input), 10x1 np array (labels, 0-9 depending on the array)

In [8]:
net = network.Network([784, 30, 10])

In [9]:
net.sizes 
# neural network sizes: 3 layers, input layer 784 neurons, output 10 neurons, and 2nd layer 30 neurons

[784, 30, 10]

In [10]:
net.weights[0].shape
#these are the weights for the 2nd layer 30 neurons, each input neuron has a weight, total of 784 input neurons

(30, 784)

In [11]:
net.weights[1].shape
#3rd layer

(10, 30)

In [12]:
net.biases[0].shape
# 1 bias for each 2nd layer neurons, 30 in total

(30, 1)

In [13]:
net.biases[1].shape

(10, 1)

I suspect this is not using gpus at all judging from the speed

In [ ]:
%timeit net.SGD(training_data, 30, 10, 3.0, test_data=test_data) # stochastic gradient descent

Epoch 0: 9415 / 10000
Epoch 1: 9427 / 10000
Epoch 2: 9458 / 10000
Epoch 3: 9443 / 10000
Epoch 4: 9443 / 10000
Epoch 5: 9488 / 10000
Epoch 6: 9476 / 10000
Epoch 7: 9461 / 10000
Epoch 8: 9491 / 10000
Epoch 9: 9470 / 10000
Epoch 10: 9501 / 10000
Epoch 11: 9453 / 10000
Epoch 12: 9476 / 10000
Epoch 13: 9501 / 10000
Epoch 14: 9485 / 10000
Epoch 15: 9490 / 10000
Epoch 16: 9472 / 10000
Epoch 17: 9495 / 10000
Epoch 18: 9513 / 10000
Epoch 19: 9483 / 10000
Epoch 20: 9459 / 10000
Epoch 21: 9471 / 10000
Epoch 22: 9484 / 10000
Epoch 23: 9501 / 10000
Epoch 24: 9489 / 10000
Epoch 25: 9489 / 10000
Epoch 26: 9504 / 10000
Epoch 27: 9471 / 10000
Epoch 28: 9489 / 10000
Epoch 29: 9503 / 10000
Epoch 0: 9506 / 10000
Epoch 1: 9469 / 10000
Epoch 2: 9499 / 10000


it is important to choose the right hyper parameters, for example learning rate eta, layers and neurons in each layer. use mini batches to speed up the process. this is called stochastic gradient descend. epoches is the number of iterations we run the learning over the entire training data.

increase the neurons to 100 from 30,

In [48]:
net = network.Network([784, 10]) # try only two layer net

In [49]:
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 5671 / 10000
Epoch 1: 5696 / 10000
Epoch 2: 5747 / 10000
Epoch 3: 5741 / 10000
Epoch 4: 5741 / 10000
Epoch 5: 5784 / 10000
Epoch 6: 5766 / 10000
Epoch 7: 5783 / 10000
Epoch 8: 5799 / 10000
Epoch 9: 5811 / 10000
Epoch 10: 5834 / 10000
Epoch 11: 5830 / 10000
Epoch 12: 5822 / 10000
Epoch 13: 5822 / 10000
Epoch 14: 5832 / 10000
Epoch 15: 5854 / 10000
Epoch 16: 5841 / 10000
Epoch 17: 5848 / 10000
Epoch 18: 5847 / 10000
Epoch 19: 5871 / 10000
Epoch 20: 5846 / 10000
Epoch 21: 5857 / 10000
Epoch 22: 5832 / 10000
Epoch 23: 5861 / 10000
Epoch 24: 5888 / 10000
Epoch 25: 5880 / 10000
Epoch 26: 5894 / 10000
Epoch 27: 5890 / 10000
Epoch 28: 5891 / 10000
Epoch 29: 5870 / 10000


In [6]:
def svm_baseline():
    training_data, validation_data, test_data = mnist_loader.load_data()
    # train
    clf = svm.SVC()
    clf.fit(training_data[0], training_data[1])
    # test
    predictions = [int(a) for a in clf.predict(test_data[0])]
    num_correct = sum(int(a == y) for a, y in zip(predictions, test_data[1]))
    print "Baseline classifier using an SVM."
    print "%s of %s values correct." % (num_correct, len(test_data[1]))

In [7]:
svm_baseline()

<gzip open file '../data/mnist.pkl.gz', mode 'rb' at 0x7f0cc29faf60 0x7f0cc2a7bfd0>
Baseline classifier using an SVM.
9435 of 10000 values correct.


backpropagation

We'll use $w^l_{jk}$ to denote the weight for the connection from the kthkth neuron in the (l−1)th(l−1)th layer to the jthjth neuron in the lthlth laye

use network2: improved accuracy by using cross entropy

In [9]:
import network2

In [13]:

training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

<gzip open file '../data/mnist.pkl.gz', mode 'rb' at 0x7f0cbbff6b70 0x7f0cbbfea210>


In [10]:
net = network2.Network([784, 30, 10], cost=network2.CrossEntropyCost)

In [11]:
net.large_weight_initializer()

In [14]:
net.SGD(training_data, 30, 10, 0.5, evaluation_data=test_data, monitor_evaluation_accuracy=True)

Epoch 0 training complete
Accuracy on evaluation data: 9105 / 10000

Epoch 1 training complete
Accuracy on evaluation data: 9261 / 10000

Epoch 2 training complete
Accuracy on evaluation data: 9321 / 10000

Epoch 3 training complete
Accuracy on evaluation data: 9327 / 10000

Epoch 4 training complete
Accuracy on evaluation data: 9377 / 10000

Epoch 5 training complete
Accuracy on evaluation data: 9383 / 10000

Epoch 6 training complete
Accuracy on evaluation data: 9390 / 10000

Epoch 7 training complete
Accuracy on evaluation data: 9434 / 10000

Epoch 8 training complete
Accuracy on evaluation data: 9406 / 10000

Epoch 9 training complete
Accuracy on evaluation data: 9457 / 10000

Epoch 10 training complete
Accuracy on evaluation data: 9423 / 10000

Epoch 11 training complete
Accuracy on evaluation data: 9461 / 10000

Epoch 12 training complete
Accuracy on evaluation data: 9475 / 10000

Epoch 13 training complete
Accuracy on evaluation data: 9463 / 10000

Epoch 14 training complete
Acc

([],
 [9105,
  9261,
  9321,
  9327,
  9377,
  9383,
  9390,
  9434,
  9406,
  9457,
  9423,
  9461,
  9475,
  9463,
  9450,
  9471,
  9448,
  9460,
  9473,
  9448,
  9468,
  9484,
  9469,
  9486,
  9463,
  9480,
  9465,
  9486,
  9503,
  9493],
 [],
 [])

demonstate generalization problems. overfitting

In [15]:
net.SGD(training_data[:1000], 400, 10, 0.5, evaluation_data=test_data, monitor_evaluation_accuracy=True, monitor_training_cost=True)

Epoch 0 training complete
Cost on training data: 0.158647893203
Accuracy on evaluation data: 9490 / 10000

Epoch 1 training complete
Cost on training data: 0.111898428845
Accuracy on evaluation data: 9505 / 10000

Epoch 2 training complete
Cost on training data: 0.101550696152
Accuracy on evaluation data: 9503 / 10000

Epoch 3 training complete
Cost on training data: 0.0931279807174
Accuracy on evaluation data: 9505 / 10000

Epoch 4 training complete
Cost on training data: 0.0875255105109
Accuracy on evaluation data: 9500 / 10000

Epoch 5 training complete
Cost on training data: 0.0840352147014
Accuracy on evaluation data: 9508 / 10000

Epoch 6 training complete
Cost on training data: 0.0782014530866
Accuracy on evaluation data: 9490 / 10000

Epoch 7 training complete
Cost on training data: 0.0747591911063
Accuracy on evaluation data: 9497 / 10000

Epoch 8 training complete
Cost on training data: 0.0727841914096
Accuracy on evaluation data: 9496 / 10000

Epoch 9 training complete
Cost 

([],
 [9490,
  9505,
  9503,
  9505,
  9500,
  9508,
  9490,
  9497,
  9496,
  9492,
  9495,
  9489,
  9498,
  9496,
  9489,
  9478,
  9485,
  9486,
  9492,
  9480,
  9484,
  9479,
  9487,
  9490,
  9493,
  9494,
  9488,
  9488,
  9482,
  9490,
  9482,
  9478,
  9480,
  9486,
  9478,
  9478,
  9476,
  9473,
  9481,
  9474,
  9472,
  9475,
  9474,
  9475,
  9476,
  9474,
  9466,
  9471,
  9476,
  9473,
  9474,
  9470,
  9469,
  9463,
  9469,
  9467,
  9469,
  9467,
  9465,
  9467,
  9466,
  9465,
  9465,
  9467,
  9468,
  9465,
  9466,
  9466,
  9463,
  9465,
  9468,
  9466,
  9463,
  9471,
  9465,
  9469,
  9465,
  9468,
  9467,
  9466,
  9467,
  9467,
  9466,
  9468,
  9465,
  9463,
  9463,
  9465,
  9465,
  9462,
  9464,
  9463,
  9462,
  9460,
  9460,
  9461,
  9460,
  9462,
  9463,
  9461,
  9463,
  9460,
  9463,
  9459,
  9459,
  9461,
  9460,
  9461,
  9462,
  9460,
  9459,
  9461,
  9457,
  9458,
  9456,
  9456,
  9456,
  9457,
  9454,
  9456,
  9453,
  9455,
  9454,
  9455,
  9

In [46]:
# deep network experimentations, 
# first a 1 hidden layer network with input 784 neurons, 30 neurons in hidden layer, and 10 output neurons.
import network2
net = network2.Network([784, 30, 10])
net.SGD(training_data, 30, 10, 0.1, lmbda=5.0, evaluation_data=validation_data, monitor_evaluation_accuracy=True)

Epoch 0 training complete
Accuracy on evaluation data: 9289 / 10000

Epoch 1 training complete
Accuracy on evaluation data: 9401 / 10000

Epoch 2 training complete
Accuracy on evaluation data: 9496 / 10000

Epoch 3 training complete
Accuracy on evaluation data: 9516 / 10000

Epoch 4 training complete
Accuracy on evaluation data: 9560 / 10000

Epoch 5 training complete
Accuracy on evaluation data: 9572 / 10000

Epoch 6 training complete
Accuracy on evaluation data: 9597 / 10000

Epoch 7 training complete
Accuracy on evaluation data: 9592 / 10000

Epoch 8 training complete
Accuracy on evaluation data: 9616 / 10000

Epoch 9 training complete
Accuracy on evaluation data: 9600 / 10000

Epoch 10 training complete
Accuracy on evaluation data: 9626 / 10000

Epoch 11 training complete
Accuracy on evaluation data: 9641 / 10000

Epoch 12 training complete
Accuracy on evaluation data: 9651 / 10000

Epoch 13 training complete
Accuracy on evaluation data: 9638 / 10000

Epoch 14 training complete
Acc

([],
 [9289,
  9401,
  9496,
  9516,
  9560,
  9572,
  9597,
  9592,
  9616,
  9600,
  9626,
  9641,
  9651,
  9638,
  9622,
  9628,
  9660,
  9648,
  9655,
  9675,
  9630,
  9644,
  9679,
  9660,
  9649,
  9659,
  9659,
  9681,
  9671,
  9656],
 [],
 [])

In [47]:
# now add another hidden layer also 30 neurons
net = network2.Network([784, 30, 30, 10])
net.SGD(training_data, 30, 10, 0.1, lmbda=5.0, evaluation_data=validation_data, monitor_evaluation_accuracy=True)

Epoch 0 training complete
Accuracy on evaluation data: 9224 / 10000

Epoch 1 training complete
Accuracy on evaluation data: 9413 / 10000

Epoch 2 training complete
Accuracy on evaluation data: 9527 / 10000

Epoch 3 training complete
Accuracy on evaluation data: 9581 / 10000

Epoch 4 training complete
Accuracy on evaluation data: 9594 / 10000

Epoch 5 training complete
Accuracy on evaluation data: 9634 / 10000

Epoch 6 training complete
Accuracy on evaluation data: 9651 / 10000

Epoch 7 training complete
Accuracy on evaluation data: 9656 / 10000

Epoch 8 training complete
Accuracy on evaluation data: 9686 / 10000

Epoch 9 training complete
Accuracy on evaluation data: 9663 / 10000

Epoch 10 training complete
Accuracy on evaluation data: 9655 / 10000

Epoch 11 training complete
Accuracy on evaluation data: 9658 / 10000

Epoch 12 training complete
Accuracy on evaluation data: 9667 / 10000

Epoch 13 training complete
Accuracy on evaluation data: 9660 / 10000

Epoch 14 training complete
Acc

([],
 [9224,
  9413,
  9527,
  9581,
  9594,
  9634,
  9651,
  9656,
  9686,
  9663,
  9655,
  9658,
  9667,
  9660,
  9677,
  9686,
  9656,
  9701,
  9674,
  9682,
  9693,
  9686,
  9680,
  9677,
  9696,
  9687,
  9700,
  9703,
  9708,
  9698],
 [],
 [])

add one more layer of 30 neurons helped. but adding another 1 or 2 more layers actuarlly make the predictions worse. why?

In [2]:
import network3
from network3 import Network
from network3 import ConvPoolLayer, FullyConnectedLayer, SoftmaxLayer

Trying to run under a GPU.  If this is not desired, then modify network3.py
to set the GPU flag to False.


In [51]:
training_data, validation_data, test_data = network3.load_data_shared()
mini_batch_size = 10
net = Network([FullyConnectedLayer(n_in=784, n_out=100), SoftmaxLayer(n_in=100, n_out=10)], mini_batch_size)
net.SGD(training_data, 60, mini_batch_size, 0.1, validation_data, test_data)

Trying to run under a GPU.  If this is not desired, then modify network3.py
to set the GPU flag to False.
Training mini-batch number 0
Training mini-batch number 1000
Training mini-batch number 2000
Training mini-batch number 3000
Training mini-batch number 4000
Epoch 0: validation accuracy 92.63%
This is the best validation accuracy to date.
The corresponding test accuracy is 91.81%
Training mini-batch number 5000
Training mini-batch number 6000
Training mini-batch number 7000
Training mini-batch number 8000
Training mini-batch number 9000
Epoch 1: validation accuracy 94.60%
This is the best validation accuracy to date.
The corresponding test accuracy is 93.97%
Training mini-batch number 10000
Training mini-batch number 11000
Training mini-batch number 12000
Training mini-batch number 13000
Training mini-batch number 14000
Epoch 2: validation accuracy 95.62%
This is the best validation accuracy to date.
The corresponding test accuracy is 95.12%
Training mini-batch number 15000
Trainin

In [5]:
# add a convolution layer use 5x5 filters, 20 features, 24x24 and a maxpooling layer 
training_data, validation_data, test_data = network3.load_data_shared()
mini_batch_size = 10
net = Network([
        ConvPoolLayer(image_shape=(mini_batch_size, 1, 28, 28), 
                      filter_shape=(20, 1, 5, 5), 
                      poolsize=(2, 2)),
        FullyConnectedLayer(n_in=20*12*12, n_out=100),
        SoftmaxLayer(n_in=100, n_out=10)], mini_batch_size)
net.SGD(training_data, 60, mini_batch_size, 0.1, 
            validation_data, test_data)

Training mini-batch number 0
Training mini-batch number 1000
Training mini-batch number 2000
Training mini-batch number 3000
Training mini-batch number 4000
Epoch 0: validation accuracy 94.22%
This is the best validation accuracy to date.
The corresponding test accuracy is 93.67%
Training mini-batch number 5000
Training mini-batch number 6000
Training mini-batch number 7000
Training mini-batch number 8000
Training mini-batch number 9000
Epoch 1: validation accuracy 96.28%
This is the best validation accuracy to date.
The corresponding test accuracy is 95.56%
Training mini-batch number 10000
Training mini-batch number 11000
Training mini-batch number 12000
Training mini-batch number 13000
Training mini-batch number 14000
Epoch 2: validation accuracy 96.96%
This is the best validation accuracy to date.
The corresponding test accuracy is 96.51%
Training mini-batch number 15000
Training mini-batch number 16000
Training mini-batch number 17000
Training mini-batch number 18000
Training mini-

In [6]:
# add in a second convolution layer
net = Network([
        ConvPoolLayer(image_shape=(mini_batch_size, 1, 28, 28), 
                      filter_shape=(20, 1, 5, 5), 
                      poolsize=(2, 2)),
        ConvPoolLayer(image_shape=(mini_batch_size, 20, 12, 12), 
                      filter_shape=(40, 20, 5, 5), 
                      poolsize=(2, 2)),
        FullyConnectedLayer(n_in=40*4*4, n_out=100),
        SoftmaxLayer(n_in=100, n_out=10)], mini_batch_size)
net.SGD(training_data, 60, mini_batch_size, 0.1, 
            validation_data, test_data)       

Training mini-batch number 0
Training mini-batch number 1000
Training mini-batch number 2000
Training mini-batch number 3000
Training mini-batch number 4000
Epoch 0: validation accuracy 86.55%
This is the best validation accuracy to date.
The corresponding test accuracy is 85.52%
Training mini-batch number 5000
Training mini-batch number 6000
Training mini-batch number 7000
Training mini-batch number 8000
Training mini-batch number 9000
Epoch 1: validation accuracy 96.50%
This is the best validation accuracy to date.
The corresponding test accuracy is 96.42%
Training mini-batch number 10000
Training mini-batch number 11000
Training mini-batch number 12000
Training mini-batch number 13000
Training mini-batch number 14000
Epoch 2: validation accuracy 97.60%
This is the best validation accuracy to date.
The corresponding test accuracy is 97.52%
Training mini-batch number 15000
Training mini-batch number 16000
Training mini-batch number 17000
Training mini-batch number 18000
Training mini-

In [7]:
# use rectified linear unit instead of sigmoid activation function or tanh function, achieved better accuracy
from network3 import ReLU
net = Network([
        ConvPoolLayer(image_shape=(mini_batch_size, 1, 28, 28), 
                      filter_shape=(20, 1, 5, 5), 
                      poolsize=(2, 2), 
                      activation_fn=ReLU),
        ConvPoolLayer(image_shape=(mini_batch_size, 20, 12, 12), 
                      filter_shape=(40, 20, 5, 5), 
                      poolsize=(2, 2), 
                      activation_fn=ReLU),
        FullyConnectedLayer(n_in=40*4*4, n_out=100, activation_fn=ReLU),
        SoftmaxLayer(n_in=100, n_out=10)], mini_batch_size)
net.SGD(training_data, 60, mini_batch_size, 0.03, 
            validation_data, test_data, lmbda=0.1)

Training mini-batch number 0
Training mini-batch number 1000
Training mini-batch number 2000
Training mini-batch number 3000
Training mini-batch number 4000
Epoch 0: validation accuracy 97.32%
This is the best validation accuracy to date.
The corresponding test accuracy is 96.68%
Training mini-batch number 5000
Training mini-batch number 6000
Training mini-batch number 7000
Training mini-batch number 8000
Training mini-batch number 9000
Epoch 1: validation accuracy 97.92%
This is the best validation accuracy to date.
The corresponding test accuracy is 97.65%
Training mini-batch number 10000
Training mini-batch number 11000
Training mini-batch number 12000
Training mini-batch number 13000
Training mini-batch number 14000
Epoch 2: validation accuracy 98.14%
This is the best validation accuracy to date.
The corresponding test accuracy is 98.17%
Training mini-batch number 15000
Training mini-batch number 16000
Training mini-batch number 17000
Training mini-batch number 18000
Training mini-

In [9]:
f = '/projects/trans_scratch/validations/workspace/szong/deep_learning/neural-networks-and-deep-learning/data/mnist.pkl.gz'
mini_batch_size = 10
expanded_training_data, _, _ = network3.load_data_shared(f)
net = Network([
        ConvPoolLayer(image_shape=(mini_batch_size, 1, 28, 28), 
                      filter_shape=(20, 1, 5, 5), 
                      poolsize=(2, 2), 
                      activation_fn=ReLU),
        ConvPoolLayer(image_shape=(mini_batch_size, 20, 12, 12), 
                      filter_shape=(40, 20, 5, 5), 
                      poolsize=(2, 2), 
                      activation_fn=ReLU),
        FullyConnectedLayer(n_in=40*4*4, n_out=100, activation_fn=ReLU),
        SoftmaxLayer(n_in=100, n_out=10)], mini_batch_size)
net.SGD(expanded_training_data, 60, mini_batch_size, 0.03, 
            validation_data, test_data, lmbda=0.1)

Training mini-batch number 0
Training mini-batch number 1000
Training mini-batch number 2000
Training mini-batch number 3000
Training mini-batch number 4000
Epoch 0: validation accuracy 97.19%
This is the best validation accuracy to date.
The corresponding test accuracy is 96.89%
Training mini-batch number 5000
Training mini-batch number 6000
Training mini-batch number 7000
Training mini-batch number 8000
Training mini-batch number 9000
Epoch 1: validation accuracy 97.92%
This is the best validation accuracy to date.
The corresponding test accuracy is 97.91%
Training mini-batch number 10000
Training mini-batch number 11000
Training mini-batch number 12000
Training mini-batch number 13000
Training mini-batch number 14000
Epoch 2: validation accuracy 98.24%
This is the best validation accuracy to date.
The corresponding test accuracy is 98.35%
Training mini-batch number 15000
Training mini-batch number 16000
Training mini-batch number 17000
Training mini-batch number 18000
Training mini-

In [11]:
# add in another fully connected layer
net = Network([
        ConvPoolLayer(image_shape=(mini_batch_size, 1, 28, 28), 
                      filter_shape=(20, 1, 5, 5), 
                      poolsize=(2, 2), 
                      activation_fn=ReLU),
        ConvPoolLayer(image_shape=(mini_batch_size, 20, 12, 12), 
                      filter_shape=(40, 20, 5, 5), 
                      poolsize=(2, 2), 
                      activation_fn=ReLU),
        FullyConnectedLayer(n_in=40*4*4, n_out=100, activation_fn=ReLU),
        FullyConnectedLayer(n_in=100, n_out=100, activation_fn=ReLU),
        SoftmaxLayer(n_in=100, n_out=10)], mini_batch_size)
%time net.SGD(expanded_training_data, 60, mini_batch_size, 0.03, validation_data, test_data, lmbda=0.1)

Training mini-batch number 0
Training mini-batch number 1000
Training mini-batch number 2000
Training mini-batch number 3000
Training mini-batch number 4000
Epoch 0: validation accuracy 97.73%
This is the best validation accuracy to date.
The corresponding test accuracy is 97.48%
Training mini-batch number 5000
Training mini-batch number 6000
Training mini-batch number 7000
Training mini-batch number 8000
Training mini-batch number 9000
Epoch 1: validation accuracy 98.14%
This is the best validation accuracy to date.
The corresponding test accuracy is 98.04%
Training mini-batch number 10000
Training mini-batch number 11000
Training mini-batch number 12000
Training mini-batch number 13000
Training mini-batch number 14000
Epoch 2: validation accuracy 98.45%
This is the best validation accuracy to date.
The corresponding test accuracy is 98.53%
Training mini-batch number 15000
Training mini-batch number 16000
Training mini-batch number 17000
Training mini-batch number 18000
Training mini-

In [12]:
# apply dropout to avoid overfitting
net = Network([
        ConvPoolLayer(image_shape=(mini_batch_size, 1, 28, 28), 
                      filter_shape=(20, 1, 5, 5), 
                      poolsize=(2, 2), 
                      activation_fn=ReLU),
        ConvPoolLayer(image_shape=(mini_batch_size, 20, 12, 12), 
                      filter_shape=(40, 20, 5, 5), 
                      poolsize=(2, 2), 
                      activation_fn=ReLU),
        FullyConnectedLayer(
            n_in=40*4*4, n_out=1000, activation_fn=ReLU, p_dropout=0.5),
        FullyConnectedLayer(
            n_in=1000, n_out=1000, activation_fn=ReLU, p_dropout=0.5),
        SoftmaxLayer(n_in=1000, n_out=10, p_dropout=0.5)], 
    mini_batch_size)
%time net.SGD(expanded_training_data, 40, mini_batch_size, 0.03, validation_data, test_data)


Training mini-batch number 0
Training mini-batch number 1000
Training mini-batch number 2000
Training mini-batch number 3000
Training mini-batch number 4000
Epoch 0: validation accuracy 96.09%
This is the best validation accuracy to date.
The corresponding test accuracy is 95.88%
Training mini-batch number 5000
Training mini-batch number 6000
Training mini-batch number 7000
Training mini-batch number 8000
Training mini-batch number 9000
Epoch 1: validation accuracy 97.64%
This is the best validation accuracy to date.
The corresponding test accuracy is 97.39%
Training mini-batch number 10000
Training mini-batch number 11000
Training mini-batch number 12000
Training mini-batch number 13000
Training mini-batch number 14000
Epoch 2: validation accuracy 98.13%
This is the best validation accuracy to date.
The corresponding test accuracy is 98.04%
Training mini-batch number 15000
Training mini-batch number 16000
Training mini-batch number 17000
Training mini-batch number 18000
Training mini-